In [1]:
import os
import numpy as np
import pandas as pd
import spacy
from tqdm import tqdm

DATA_DIR="data/cleaned/"

In [2]:

# df_langs=pd.read_csv(DATA_DIR+"langs.csv")
# df_langs=df_langs.set_index('id')
df_train=pd.read_pickle(DATA_DIR+"train_cleaned.pkl")
df_train=df_train.set_index('id')
df_train.head()

,title,link,description,long_description
id,,,,
2674,ପ୍ରବଳ ଶୀତରେ ଥରୁଛି ସାରା ଓଡ଼ିଶା । ୧୩ଟି ଜିଲ୍ଲା ପାଇ...,https://www.newspointapp.com/odia-news/publish...,କନକ ବ୍ୟୁରୋ : ଜାଡରେ ଥରୁଛି ସାରା ଓଡିଶା । ସ୍ୱାଭାବି...,କନକ ବ୍ୟୁରୋ : ଜାଡରେ ଥରୁଛି ସାରା ଓଡିଶା । ସ୍ୱାଭାବି...
3874,'संघर्षशील और जुझारू संस्कृतिकर्मी थे संदीपन व...,https://navbharattimes.indiatimes.com/metro/lu...,\Bसंदीपन ने कलाकारों को दिशा देने के साथ नाट्य...,B Sandipan awakened the play with consciousnes...
425,डीपी रामभरोसे,https://maharashtratimes.indiatimes.com/citize...,NaN,NaN
5177,Whipping up nationalism has a shelf life: Kama...,https://www.newspointapp.com/english-news/publ...,BHOPAL: The widespread protests against CAA ar...,BHOPAL: The widespread protests against CAA ar...
3260,முலாயம் சிங் மருத்துவமனையில் அனுமதி,https://www.seithisolai.com/mulayam-singh-admi...,சமாஜ்வாதி கட்சியின் நிறுவனத் தலைவர் முலாயம் சி...,Samajwadi Party founder Mulayam Singh has been...


In [17]:
len(df_train)

1411103

In [16]:
list(range(0,len(df_train), len(df_train)//4))

[0, 352775, 705550, 1058325, 1411100]

### sentences.to_csv()

In [133]:
import re

nlp=spacy.load('en_core_web_sm')

tagged_sentences=[]
selected_phrases=[]
proper_nouns=[]

cols=['title', 'description', 'long_description']
lang_cols=[i+"_lang" for i in cols]
df_train[cols][df_langs[lang_cols]!='en']=None

sentences=df_train['long_description']
sentences.fillna(df_train['description'], inplace=True)
sentences.fillna(df_train['title'], inplace=True)

considered_tags = set(['NN', 'NNS', 'NNP', 'NNPS', 'JJ'])
propnoun_tags=set(['NNP', 'NNPS'])

def tokenize(sent):
    ret_l=[]
    for token in nlp(sent):
        ret_l.append((token.text, token.tag_,
                      #token.lemma_
                     ))
    return ret_l

def extract_phrases(sent):
    keyphrase_candidate = set()

    np_parser = nltk.RegexpParser("""  NP:
        {<NN.*|JJ>*<NN.*>}  # Adjective(s)(optional) + Noun(s)""")  # Noun phrase parser
    tree = np_parser.parse(sent)  # Generator with one tree per sentence

    for subtree in tree.subtrees(filter=lambda t: t.label() == 'NP'):  # For each nounphrase
        # Concatenate the token with a space
        keyphrase_candidate.add(' '.join(word for word, tag in subtree.leaves()))

    keyphrase_candidate = {kp for kp in keyphrase_candidate if len(kp.split()) <= 5}

    keyphrase_candidate = list(keyphrase_candidate)
    return keyphrase_candidate

def extract_candidates(sent):
    ret_l=[]
    propnoun_flag=False
    propnouns_l=[]
    temp=[]
    for token in sent:
        if token[1] in propnoun_tags:
            if propnoun_flag:
                temp.append(token[0])
            else:
                temp=[token[0]]
                propnoun_flag=True
        elif propnoun_flag:
            propnouns_l.append(" ".join(temp))
            propnoun_flag=False
        if token[1] in considered_tags and len(token[0])>3:
            ret_l.append(token[0])
    return np.unique(ret_l), np.unique(propnouns_l)
            
for sentence in tqdm(sentences):
    #" ".join(simple_preprocess(sentence))
    sentence=re.sub("[\\\\\'\"]", "", sentence.lower())
    tokenized=tokenize(sentence)
    tagged_sentences.append(tokenized)
    
#     sel_l, pro_l=extract_candidates(tokenized)
#     selected_phrases.append(sel_l)
#     proper_nouns.append(pro_l)
    selected_phrases.append(extract_phrases(tokenized))

/home/darp_lord/Installs/anaconda3/envs/NLP/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
100%|██████████| 100/100 [00:04<00:00, 21.49it/s]


In [136]:
selected_phrases[3]

['hapur',
 'nbt news',
 'charge',
 'mahavir singh',
 'tuesday',
 'body',
 'forest department',
 'national highway-9',
 'police station pilkhuwa area',
 'inspector',
 'reindeer',
 'truck',
 'station',
 'police']

In [11]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def get_topk(doc, X, candidates, N=10, beta=0.65, alias_threshold=0.8):
    N=min(N, len(candidates))
    doc_sim=cosine_similarity(X, doc)
    doc_sim_norm = doc_sim/np.max(doc_sim)
    doc_sim_norm = 0.5 + (doc_sim_norm - np.average(doc_sim_norm)) / np.std(doc_sim_norm)
    sim_between = cosine_similarity(X)
    np.fill_diagonal(sim_between, np.NaN)
    sim_between_norm = sim_between/np.nanmax(sim_between, axis=0)
    sim_between_norm = 0.5 + (sim_between_norm - np.nanmean(sim_between_norm, axis=0)) / np.nanstd(sim_between_norm, axis=0)
    selected_candidates = []
    unselected_candidates = [c for c in range(len(X))]
    j = np.argmax(doc_sim)
    selected_candidates.append(j)
    unselected_candidates.remove(j)
    for _ in range(N-1):
        selec_array = np.array(selected_candidates)
        unselec_array = np.array(unselected_candidates)
        distance_to_doc = doc_sim_norm[unselec_array, :]
        dist_between = sim_between_norm[unselec_array][:, selec_array]
        if dist_between.ndim == 1:
            dist_between = dist_between[:, np.newaxis]
        j = np.argmax(beta * distance_to_doc - (1 - beta) * np.max(dist_between, axis=1).reshape(-1, 1))
        item_idx = unselected_candidates[j]
        selected_candidates.append(item_idx)
        unselected_candidates.remove(item_idx)
    return candidates[selected_candidates].tolist()